<a href="https://colab.research.google.com/github/Tclack88/MountainProject/blob/master/mproj_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import os
import re
import math
import pandas as pd


In [0]:
url = "https://www.mountainproject.com/data/get-routes?routeIds=107987020,105928340,105930602,105937715,105928356,105823427,105965021,106814369,105825541,105878937,106837289,105878940,105822594,106802728,105825537,106837320,106809384,105964992,105823585,105823589,106837304,105977103,105822598,105837277,105965166,114944809,105805488,105805463,105805473,105930611,105874919,105805492,105807203,105807210,106480652,105874909,105874928,105930621,105874923,105807964,105926384,105926390,105807952,105926380,105989753,105807974,106005766,105878944,105878952,105878947,105986456,105986463,105977112,105986444,110964122,105852036,105977129,106521533,105852041,105852049,107985763,105977106,105977117,105930633,105977120,105852052,106194447,106802720,105900497,106837985,106256560,114536121,105852331,105881435,106177038,105881432,106177033,106005775,106177023,106005771,106444598,105977097,106256569,106444589,105987370,105931048,105930851,107224078,106809677,105930861,106239097,106840256,105930799,106840238,114536109,114534895,105931043,105933455,114534865,105985052&key=109791883-f50800e35d54ebe7eb9bd5ea68849f1d"

request = requests.get(url)

In [0]:



def flatten(lst):
    return sum( ([x] if not isinstance(x, list) else flatten(x) for x in lst), [] )
  
  
  

def find_sub_areas(url):
    source = requests.get(url).text
    soup = BeautifulSoup(source,'html.parser')
    sublinks = []
    lef_navs = soup.find_all('div',class_='lef-nav-row')
    if len(lef_navs) == 0:
        return find_routes(url) # We are at the lowest and return just route links
    for lef_nav in lef_navs:
        links = lef_nav.findChildren('a')
        sublinks.append([link['href'] for link in links])
    #print("-"*80)
    #print("\nnumber of sub areas:",len(lef_navs))
    #print("\nhere are the links:")
    sublinks = flatten(sublinks)
    #print(sublinks)
    #print("#"*20)
    deeper_sublinks = []
    for sublink in sublinks:
        #sublinks.remove(sublink)
        deeper_sublink = find_sub_areas(sublink)
        #print("\ndeeper_sublinks:")
        #print(deeper_sublink)
        deeper_sublinks.append(deeper_sublink)
    return deeper_sublinks



def find_routes(url):
    route_links = []
    source = requests.get(url).text
    soup = BeautifulSoup(source,'html.parser')
    routes = soup.find('table',id='left-nav-route-table')
    routes = routes.findChildren('a')
    route_links.append([route['href'] for route in routes])
    route_links = flatten(route_links)
    return route_links

  
  
def group_routes(route_ids,n=100):
    for ids in range(0,len(route_ids),n):
        yield (route_ids[ids:ids+n])   



def create_dataframe(json):
    df = pd.DataFrame(json)
    df['route'] = df.name.astype(str)+'-'+df.type.astype(str)+\
    '-'+df.rating.astype(str)+'-'+df.pitches.astype(str)+'p'
    cols = ['route','location','longitude','latitude']
    df = df[cols]
    df.location = df.location.apply(lambda x: '-'.join(x))
    #location = list(df.location)
    #location = '-'.join(df.location)
    #df = df.iloc[0,:]
    #df.location = location
    print("DF:\n",df)
    return df

  
  

def request_routes(route_groups):
    dfs = []
    for route_group in route_groups:
        route_group = ','.join(route_group)
        url_start = "https://www.mountainproject.com/data/get-routes?routeIds="
        url_end = "&key=109791883-f50800e35d54ebe7eb9bd5ea68849f1d" # my MP key
        request_string = url_start + route_group + url_end
        request = requests.get(request_string)
        json = request.json()['routes']
        print("JSON:\n",json)
        df = create_dataframe(json)
        dfs.append(df)
    return dfs


In [28]:
url = "https://www.mountainproject.com/area/105931166/central-pinnacles"

all_routes = find_sub_areas(url)

all_routes = flatten(all_routes)
print(all_routes)
print('\n^',len(all_routes),"routes in total")

['https://www.mountainproject.com/route/107987020/coolie-crank', 'https://www.mountainproject.com/route/105928340/daves-crack', 'https://www.mountainproject.com/route/105930602/grain-scoop', 'https://www.mountainproject.com/route/105937715/oso-paws-left', 'https://www.mountainproject.com/route/105928356/target-practice', 'https://www.mountainproject.com/route/105823427/better-luck-next-time', 'https://www.mountainproject.com/route/105965021/blue-sky-mine', 'https://www.mountainproject.com/route/106814369/bobbing-for-ear-snax', 'https://www.mountainproject.com/route/105825541/brewed-awakening', 'https://www.mountainproject.com/route/105878937/bum-steer', 'https://www.mountainproject.com/route/106837289/bye-chimney', 'https://www.mountainproject.com/route/105878940/chaps-my-hide', 'https://www.mountainproject.com/route/105822594/claim-jumper', 'https://www.mountainproject.com/route/106802728/colonial-ear-wax', 'https://www.mountainproject.com/route/105825537/coyotes-in-the-henhouse', 'ht

In [0]:
route_ids = []

for route in all_routes:
    match = re.findall('\d+',route)[0]  # The first number is always the route id
                                        # If any number appear in route name, they
                                        # will be ignored (\d+ is one or more digits)
    route_ids.append(match)

    
    
route_groups = list(group_routes(route_ids))

In [57]:
dfs = request_routes(route_groups)

JSON:
 [{'id': 105805463, 'name': 'Bye Crackie', 'type': 'Sport', 'rating': '5.7', 'stars': 3.9, 'starVotes': 303, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105805463/bye-crackie', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106032952_sqsmall_1558462978.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106032952_small_1558462978.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106032952_smallMed_1558462978.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106032952_medium_1558462978.jpg', 'longitude': -116.8795, 'latitude': 34.3033}, {'id': 105805473, 'name': 'Coyotes at Sunset', 'type': 'Sport', 'rating': '5.8', 'stars': 4, 'starVotes': 313, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Ce

In [0]:
# a = [{'id': 105805463, 'name': 'Bye Crackie', 'type': 'Sport', 'rating': '5.7', 'stars': 3.9, 'starVotes': 303, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105805463/bye-crackie', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106032952_sqsmall_1558462978.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106032952_small_1558462978.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106032952_smallMed_1558462978.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106032952_medium_1558462978.jpg', 'longitude': -116.8795, 'latitude': 34.3033}, {'id': 105805473, 'name': 'Coyotes at Sunset', 'type': 'Sport', 'rating': '5.8', 'stars': 4, 'starVotes': 313, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105805473/coyotes-at-sunset', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106869997_sqsmall_1494147185.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106869997_small_1494147185.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106869997_smallMed_1494147185.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106869997_medium_1494147185.jpg', 'longitude': -116.8778, 'latitude': 34.309}, {'id': 105805488, 'name': 'Black Magic Poodle', 'type': 'Sport', 'rating': '5.9', 'stars': 3.6, 'starVotes': 132, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105805488/black-magic-poodle', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106875312_sqsmall_1494147573.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106875312_small_1494147573.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106875312_smallMed_1494147573.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106875312_medium_1494147573.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105805492, 'name': 'Golden Poodle', 'type': 'Sport', 'rating': '5.9+', 'stars': 3.7, 'starVotes': 190, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105805492/golden-poodle', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106869997_sqsmall_1494147185.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106869997_small_1494147185.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106869997_smallMed_1494147185.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106869997_medium_1494147185.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105807203, 'name': 'Golden Spike', 'type': 'Sport', 'rating': '5.10b', 'stars': 3.4, 'starVotes': 79, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105807203/golden-spike', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106869997_sqsmall_1494147185.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106869997_small_1494147185.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106869997_smallMed_1494147185.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106869997_medium_1494147185.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105807210, 'name': 'High Noon', 'type': 'Sport', 'rating': '5.10b', 'stars': 3.5, 'starVotes': 94, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105807210/high-noon', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106869997_sqsmall_1494147185.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106869997_small_1494147185.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106869997_smallMed_1494147185.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106869997_medium_1494147185.jpg', 'longitude': -116.8784, 'latitude': 34.3078}, {'id': 105807952, 'name': 'Pistol Pete', 'type': 'Sport', 'rating': '5.10a/b', 'stars': 4.5, 'starVotes': 266, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105807952/pistol-pete', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106024245_sqsmall_1558392577.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106024245_small_1558392577.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106024245_smallMed_1558392577.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106024245_medium_1558392577.jpg', 'longitude': -116.8783, 'latitude': 34.3078}, {'id': 105807964, 'name': "Doc's Holiday", 'type': 'Sport', 'rating': '5.10d', 'stars': 4.4, 'starVotes': 146, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105807964/docs-holiday', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/109378767_sqsmall_1494348461.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/109378767_small_1494348461.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/109378767_smallMed_1494348461.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/109378767_medium_1494348461.jpg', 'longitude': -116.8788, 'latitude': 34.3075}, {'id': 105807974, 'name': 'Unforgiven', 'type': 'Sport', 'rating': '5.11b', 'stars': 3.5, 'starVotes': 40, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105807974/unforgiven', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108002578_sqsmall_1494256565.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108002578_small_1494256565.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108002578_smallMed_1494256565.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108002578_medium_1494256565.jpg', 'longitude': -116.8783, 'latitude': 34.3078}, {'id': 105822594, 'name': 'Claim Jumper', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.7, 'starVotes': 201, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105822594/claim-jumper', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105822598, 'name': 'One Armed Bandit', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.9, 'starVotes': 185, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105822598/one-armed-bandit', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105823427, 'name': 'Better Luck Next Time', 'type': 'Sport', 'rating': '5.10d', 'stars': 2.3, 'starVotes': 9, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105823427/better-luck-next-time', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105970166_sqsmall_1558385069.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105970166_small_1558385069.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105970166_smallMed_1558385069.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105970166_medium_1558385069.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105823585, 'name': 'Lady Luck', 'type': 'Sport', 'rating': '5.9', 'stars': 2.8, 'starVotes': 110, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105823585/lady-luck', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105970171_sqsmall_1558385071.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105970171_small_1558385071.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105970171_smallMed_1558385071.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105970171_medium_1558385071.jpg', 'longitude': -116.8785, 'latitude': 34.308}, {'id': 105823589, 'name': 'Lucky Cuss', 'type': 'Sport', 'rating': '5.11a', 'stars': 3, 'starVotes': 19, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105823589/lucky-cuss', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105970155_sqsmall_1558385064.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105970155_small_1558385064.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105970155_smallMed_1558385064.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105970155_medium_1558385064.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105825537, 'name': 'Coyotes in the Henhouse', 'type': 'Sport', 'rating': '5.10d', 'stars': 4.6, 'starVotes': 196, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105825537/coyotes-in-the-henhouse', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105825541, 'name': 'Brewed Awakening', 'type': 'Trad', 'rating': '5.11b', 'stars': 3.5, 'starVotes': 17, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105825541/brewed-awakening', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105837277, 'name': 'The Papper', 'type': 'Sport', 'rating': '5.10c', 'stars': 3.6, 'starVotes': 32, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105837277/the-papper', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105852036, 'name': 'Drug of Choice', 'type': 'Sport', 'rating': '5.12a', 'stars': 4.1, 'starVotes': 16, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105852036/drug-of-choice', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106495524_sqsmall_1494114172.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106495524_small_1494114172.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106495524_smallMed_1494114172.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106495524_medium_1494114172.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105852041, 'name': 'Finger Crimping Good', 'type': 'Sport', 'rating': '5.11c', 'stars': 3.5, 'starVotes': 10, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105852041/finger-crimping-good', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/112147274_sqsmall_1494323739.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/112147274_small_1494323739.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/112147274_smallMed_1494323739.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/112147274_medium_1494323739.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105852049, 'name': 'Gunsmoke', 'type': 'Trad', 'rating': '5.9', 'stars': 2.7, 'starVotes': 12, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105852049/gunsmoke', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105852052, 'name': 'Smoking Gun', 'type': 'Sport', 'rating': '5.10b', 'stars': 2.3, 'starVotes': 13, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105852052/smoking-gun', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106495518_sqsmall_1494114170.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106495518_small_1494114170.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106495518_smallMed_1494114170.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106495518_medium_1494114170.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105852331, 'name': 'The Incinerator', 'type': 'Sport', 'rating': '5.12a', 'stars': 4.5, 'starVotes': 83, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Incinerator Wall'], 'url': 'https://www.mountainproject.com/route/105852331/the-incinerator', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105922062_sqsmall_1557943115.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105922062_small_1557943115.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105922062_smallMed_1557943115.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105922062_medium_1557943115.jpg', 'longitude': -116.8783, 'latitude': 34.3092}, {'id': 105874909, 'name': 'Pass the Bucket', 'type': 'Trad', 'rating': '5.8 PG13', 'stars': 3.3, 'starVotes': 8, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105874909/pass-the-bucket', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108074979_sqsmall_1494261401.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108074979_small_1494261401.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108074979_smallMed_1494261401.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108074979_medium_1494261401.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105874919, 'name': 'Es Muy Bueno', 'type': 'Trad', 'rating': '5.8', 'stars': 2.8, 'starVotes': 16, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105874919/es-muy-bueno', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108070152_sqsmall_1494261060.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108070152_small_1494261060.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108070152_smallMed_1494261060.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108070152_medium_1494261060.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105874923, 'name': 'Western Farm Service', 'type': 'Trad', 'rating': '5.8', 'stars': 2.6, 'starVotes': 24, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105874923/western-farm-service', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106497146_sqsmall_1494114360.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106497146_small_1494114360.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106497146_smallMed_1494114360.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106497146_medium_1494114360.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105874928, 'name': 'Red Brewster', 'type': 'Trad', 'rating': '5.9 R', 'stars': 2.8, 'starVotes': 4, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105874928/red-brewster', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105878937, 'name': 'Bum Steer', 'type': 'Sport', 'rating': '5.10a', 'stars': 3, 'starVotes': 103, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105878937/bum-steer', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.308}, {'id': 105878940, 'name': 'Chaps My Hide', 'type': 'Sport', 'rating': '5.11a', 'stars': 3.3, 'starVotes': 55, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105878940/chaps-my-hide', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105878944, 'name': 'Gold Bug', 'type': 'Sport', 'rating': '5.8', 'stars': 3, 'starVotes': 96, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gold Wall'], 'url': 'https://www.mountainproject.com/route/105878944/gold-bug', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/112935359_sqsmall_1495571070.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/112935359_small_1495571070.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/112935359_smallMed_1495571070.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/112935359_medium_1495571070.jpg', 'longitude': -116.8782, 'latitude': 34.3078}, {'id': 105878947, 'name': 'Hidden Gold', 'type': 'Sport', 'rating': '5.7', 'stars': 3, 'starVotes': 115, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gold Wall'], 'url': 'https://www.mountainproject.com/route/105878947/hidden-gold', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/113156070_sqsmall_1497465394.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/113156070_small_1497465394.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/113156070_smallMed_1497465394.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/113156070_medium_1497465394.jpg', 'longitude': -116.8782, 'latitude': 34.3078}, {'id': 105878952, 'name': 'Gold Standard', 'type': 'Sport', 'rating': '5.6', 'stars': 3.9, 'starVotes': 185, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gold Wall'], 'url': 'https://www.mountainproject.com/route/105878952/gold-standard', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106816848_sqsmall_1494142976.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106816848_small_1494142976.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106816848_smallMed_1494142976.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106816848_medium_1494142976.jpg', 'longitude': -116.8777, 'latitude': 34.3058}, {'id': 105881432, 'name': 'Branding Iron', 'type': 'Trad', 'rating': '5.10a', 'stars': 3, 'starVotes': 4, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/105881432/branding-iron', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 105881435, 'name': 'Black Angus', 'type': 'Trad', 'rating': '5.10a', 'stars': 2.8, 'starVotes': 4, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/105881435/black-angus', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 105900497, 'name': 'Burning Man', 'type': 'Sport', 'rating': '5.9', 'stars': 2.7, 'starVotes': 13, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Incinerator Wall'], 'url': 'https://www.mountainproject.com/route/105900497/burning-man', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107312395_sqsmall_1494180648.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107312395_small_1494180648.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107312395_smallMed_1494180648.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107312395_medium_1494180648.jpg', 'longitude': -116.8782, 'latitude': 34.3091}, {'id': 105926380, 'name': 'Quick on the Draw', 'type': 'Sport', 'rating': '5.10d', 'stars': 3.4, 'starVotes': 37, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105926380/quick-on-the-draw', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8783, 'latitude': 34.3078}, {'id': 105926384, 'name': 'Far Beyond Driven', 'type': 'Sport', 'rating': '5.10b', 'stars': 2.5, 'starVotes': 11, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105926384/far-beyond-driven', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106465197_sqsmall_1494111444.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106465197_small_1494111444.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106465197_smallMed_1494111444.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106465197_medium_1494111444.jpg', 'longitude': -116.8783, 'latitude': 34.3078}, {'id': 105926390, 'name': 'Loose Women, Loose Rock', 'type': 'Trad', 'rating': '5.9+', 'stars': 3, 'starVotes': 13, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105926390/loose-women-loose-rock', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8783, 'latitude': 34.3078}, {'id': 105928340, 'name': "Dave's Crack", 'type': 'Trad', 'rating': '5.6', 'stars': 2.5, 'starVotes': 10, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Camp Rock'], 'url': 'https://www.mountainproject.com/route/105928340/daves-crack', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107711517_sqsmall_1494206546.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107711517_small_1494206546.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107711517_smallMed_1494206546.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107711517_medium_1494206546.jpg', 'longitude': -116.879, 'latitude': 34.3072}, {'id': 105928356, 'name': 'Target Practice', 'type': 'Sport', 'rating': '5.10b', 'stars': 2.7, 'starVotes': 12, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Camp Rock'], 'url': 'https://www.mountainproject.com/route/105928356/target-practice', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107992110_sqsmall_1494255824.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107992110_small_1494255824.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107992110_smallMed_1494255824.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107992110_medium_1494255824.jpg', 'longitude': -116.879, 'latitude': 34.3072}, {'id': 105930602, 'name': 'Grain Scoop', 'type': 'TR', 'rating': '5.9', 'stars': 2.3, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Camp Rock'], 'url': 'https://www.mountainproject.com/route/105930602/grain-scoop', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/110581057_sqsmall_1494340714.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/110581057_small_1494340714.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/110581057_smallMed_1494340714.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/110581057_medium_1494340714.jpg', 'longitude': -116.879, 'latitude': 34.307}, {'id': 105930611, 'name': 'Eight Second Ride', 'type': 'Sport', 'rating': '5.8', 'stars': 2.6, 'starVotes': 22, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105930611/eight-second-ride', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106488377_sqsmall_1494113583.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106488377_small_1494113583.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106488377_smallMed_1494113583.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106488377_medium_1494113583.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105930621, 'name': 'Saddle Tramp', 'type': 'TR', 'rating': '5.9', 'stars': 2.3, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/105930621/saddle-tramp', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932611_sqsmall_1557951207.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932611_small_1557951207.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932611_smallMed_1557951207.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932611_medium_1557951207.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 105930633, 'name': 'Rawhide', 'type': 'Sport', 'rating': '5.10b', 'stars': 2.8, 'starVotes': 10, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105930633/rawhide', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/112147247_sqsmall_1494323738.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/112147247_small_1494323738.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/112147247_smallMed_1494323738.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/112147247_medium_1494323738.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105930799, 'name': 'Golden Nugget', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.7, 'starVotes': 115, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105930799/golden-nugget', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/112188476_sqsmall_1494325886.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/112188476_small_1494325886.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/112188476_smallMed_1494325886.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/112188476_medium_1494325886.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105930851, 'name': 'Blasting Cap', 'type': 'Sport', 'rating': '5.4', 'stars': 2.9, 'starVotes': 76, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105930851/blasting-cap', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/113781144_sqsmall_1509679833.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/113781144_small_1509679833.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/113781144_smallMed_1509679833.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/113781144_medium_1509679833.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105930861, 'name': 'Fire in the Hole', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.7, 'starVotes': 94, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105930861/fire-in-the-hole', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108073808_sqsmall_1494261323.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108073808_small_1494261323.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108073808_smallMed_1494261323.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108073808_medium_1494261323.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105931043, 'name': 'Motherlode', 'type': 'Sport', 'rating': '5.11b', 'stars': 3.2, 'starVotes': 28, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105931043/motherlode', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108073755_sqsmall_1494261309.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108073755_small_1494261309.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108073755_smallMed_1494261309.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108073755_medium_1494261309.jpg', 'longitude': -116.8782, 'latitude': 34.309}, {'id': 105931048, 'name': 'Black Bart', 'type': 'Sport', 'rating': '5.11a', 'stars': 3.5, 'starVotes': 28, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105931048/black-bart', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108073761_sqsmall_1494261312.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108073761_small_1494261312.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108073761_smallMed_1494261312.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108073761_medium_1494261312.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105933455, 'name': 'Out of Our Mines', 'type': 'Sport', 'rating': '5.11a/b', 'stars': 3.7, 'starVotes': 16, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105933455/out-of-our-mines', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/111042640_sqsmall_1494338514.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/111042640_small_1494338514.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/111042640_smallMed_1494338514.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/111042640_medium_1494338514.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105937715, 'name': 'Oso Paws Left', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.3, 'starVotes': 19, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Camp Rock'], 'url': 'https://www.mountainproject.com/route/105937715/oso-paws-left', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107698522_sqsmall_1494205702.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107698522_small_1494205702.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107698522_smallMed_1494205702.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107698522_medium_1494205702.jpg', 'longitude': -116.8788, 'latitude': 34.307}, {'id': 105964992, 'name': "The Hangin' Judge", 'type': 'Sport', 'rating': '5.11c/d', 'stars': 3.6, 'starVotes': 43, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105964992/the-hangin-judge', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106191296_sqsmall_1494086812.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106191296_small_1494086812.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106191296_smallMed_1494086812.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106191296_medium_1494086812.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105965021, 'name': 'Blue Sky Mine', 'type': 'Sport', 'rating': '5.10c', 'stars': 3.5, 'starVotes': 24, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105965021/blue-sky-mine', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105965166, 'name': 'Public Hanging', 'type': 'Sport', 'rating': '5.11c/d', 'stars': 3.8, 'starVotes': 59, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105965166/public-hanging', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105932635_sqsmall_1557951213.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105932635_small_1557951213.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105932635_smallMed_1557951213.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105932635_medium_1557951213.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105977097, 'name': 'Mule Kick', 'type': 'Trad', 'rating': '5.8', 'stars': 3, 'starVotes': 11, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Moonshine Dome'], 'url': 'https://www.mountainproject.com/route/105977097/mule-kick', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108228858_sqsmall_1494270629.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108228858_small_1494270629.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108228858_smallMed_1494270629.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108228858_medium_1494270629.jpg', 'longitude': -116.8789, 'latitude': 34.3091}, {'id': 105977103, 'name': 'Necktie Party', 'type': 'Sport', 'rating': '5.10d', 'stars': 3.3, 'starVotes': 13, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/105977103/necktie-party', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106851447_sqsmall_1494145597.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106851447_small_1494145597.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106851447_smallMed_1494145597.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106851447_medium_1494145597.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 105977106, 'name': 'Narcotic Prayer', 'type': 'TR', 'rating': '5.11a', 'stars': 2, 'starVotes': 1, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105977106/narcotic-prayer', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105977112, 'name': 'Black Powder', 'type': 'Trad', 'rating': '5.7', 'stars': 3, 'starVotes': 21, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105977112/black-powder', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/114800191_sqsmall_1533143956.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/114800191_small_1533143956.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/114800191_smallMed_1533143956.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/114800191_medium_1533143956.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105977117, 'name': 'Powderhorn', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.5, 'starVotes': 57, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105977117/powderhorn', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/114933984_sqsmall_1533586179.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/114933984_small_1533586179.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/114933984_smallMed_1533586179.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/114933984_medium_1533586179.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105977120, 'name': "Smokin' the Rock", 'type': 'Sport', 'rating': '5.10b', 'stars': 3.2, 'starVotes': 30, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105977120/smokin-the-rock', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106829819_sqsmall_1494143925.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106829819_small_1494143925.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106829819_smallMed_1494143925.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106829819_medium_1494143925.jpg', 'longitude': -116.8792, 'latitude': 34.3083}, {'id': 105977129, 'name': 'Fever Pitch', 'type': 'Sport', 'rating': '5.3', 'stars': 2.9, 'starVotes': 50, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105977129/fever-pitch', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/117641529_sqsmall_1566747017.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/117641529_small_1566747017.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/117641529_smallMed_1566747017.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/117641529_medium_1566747017.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105985052, 'name': 'Psychedelic Sluice', 'type': 'Trad', 'rating': '5.6', 'stars': 3.5, 'starVotes': 143, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105985052/psychedelic-sluice', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108073759_sqsmall_1494261311.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108073759_small_1494261311.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108073759_smallMed_1494261311.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108073759_medium_1494261311.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105986444, 'name': 'Cali Gold', 'type': 'TR', 'rating': '5.5', 'stars': 2.6, 'starVotes': 13, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105986444/cali-gold', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105986456, 'name': 'After the Gold Rush', 'type': 'TR', 'rating': '5.8-', 'stars': 2.8, 'starVotes': 12, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105986456/after-the-gold-rush', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/105986480_sqsmall_1558387418.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/105986480_small_1558387418.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/105986480_smallMed_1558387418.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/105986480_medium_1558387418.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 105986463, 'name': 'Bacon Taco', 'type': 'Sport', 'rating': '5.10a', 'stars': 3, 'starVotes': 50, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/105986463/bacon-taco', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/109503525_sqsmall_1494352187.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/109503525_small_1494352187.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/109503525_smallMed_1494352187.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/109503525_medium_1494352187.jpg', 'longitude': -116.8784, 'latitude': 34.3081}, {'id': 105987370, 'name': 'Belleville', 'type': 'TR', 'rating': '5.9+', 'stars': 2.5, 'starVotes': 11, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/105987370/belleville', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107617338_sqsmall_1494200450.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107617338_small_1494200450.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107617338_smallMed_1494200450.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107617338_medium_1494200450.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 105989753, 'name': 'Skyline Pillar', 'type': 'Sport', 'rating': '5.7', 'stars': 3.1, 'starVotes': 41, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Doc Holliday Wall'], 'url': 'https://www.mountainproject.com/route/105989753/skyline-pillar', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106019097_sqsmall_1558392026.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106019097_small_1558392026.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106019097_smallMed_1558392026.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106019097_medium_1558392026.jpg', 'longitude': -116.8783, 'latitude': 34.3078}, {'id': 106005766, 'name': 'Flash for Hash', 'type': 'Trad', 'rating': '5.8', 'stars': 2.7, 'starVotes': 18, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gold Wall'], 'url': 'https://www.mountainproject.com/route/106005766/flash-for-hash', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106534528_sqsmall_1494117981.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106534528_small_1494117981.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106534528_smallMed_1494117981.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106534528_medium_1494117981.jpg', 'longitude': -116.8782, 'latitude': 34.3078}, {'id': 106005771, 'name': 'Wild Kingdom', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.5, 'starVotes': 13, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/106005771/wild-kingdom', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106005774_sqsmall_1558390185.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106005774_small_1558390185.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106005774_smallMed_1558390185.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106005774_medium_1558390185.jpg', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 106005775, 'name': 'Down with the Herd', 'type': 'Sport', 'rating': '5.10d', 'stars': 4, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/106005775/down-with-the-herd', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106005781_sqsmall_1558390187.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106005781_small_1558390187.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106005781_smallMed_1558390187.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106005781_medium_1558390187.jpg', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 106177023, 'name': 'March of Dimes', 'type': 'Sport', 'rating': '5.11a', 'stars': 3.2, 'starVotes': 5, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/106177023/march-of-dimes', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 106177033, 'name': 'Cowdura', 'type': 'Trad', 'rating': '5.10b', 'stars': 3, 'starVotes': 2, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/106177033/cowdura', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 106177038, 'name': 'Bovine Eyes', 'type': 'TR', 'rating': '5.11a', 'stars': 2, 'starVotes': 1, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Mad Cow Wall'], 'url': 'https://www.mountainproject.com/route/106177038/bovine-eyes', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.878, 'latitude': 34.3079}, {'id': 106194447, 'name': 'Testosterone Crack', 'type': 'Sport', 'rating': '5.10c', 'stars': 3.6, 'starVotes': 21, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/106194447/testosterone-crack', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106196473_sqsmall_1494087197.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106196473_small_1494087197.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106196473_smallMed_1494087197.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106196473_medium_1494087197.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 106239097, 'name': 'Fools Gold', 'type': 'Trad', 'rating': '5.6', 'stars': 3, 'starVotes': 46, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/106239097/fools-gold', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/111869963_sqsmall_1494308795.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/111869963_small_1494308795.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/111869963_smallMed_1494308795.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/111869963_medium_1494308795.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 106256560, 'name': 'Fire Walker', 'type': 'Trad', 'rating': '5.10a', 'stars': 2.3, 'starVotes': 6, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Incinerator Wall'], 'url': 'https://www.mountainproject.com/route/106256560/fire-walker', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8782, 'latitude': 34.3091}, {'id': 106256569, 'name': 'Prohibited', 'type': 'Trad', 'rating': '5.7', 'stars': 2.7, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Moonshine Dome'], 'url': 'https://www.mountainproject.com/route/106256569/prohibited', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106471482_sqsmall_1494112006.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106471482_small_1494112006.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106471482_smallMed_1494112006.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106471482_medium_1494112006.jpg', 'longitude': -116.8789, 'latitude': 34.3091}, {'id': 106444589, 'name': 'Skull Cracker', 'type': 'Trad', 'rating': '5.8', 'stars': 2, 'starVotes': 5, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Moonshine Dome'], 'url': 'https://www.mountainproject.com/route/106444589/skull-cracker', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106471474_sqsmall_1494112003.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106471474_small_1494112003.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106471474_smallMed_1494112003.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106471474_medium_1494112003.jpg', 'longitude': -116.8789, 'latitude': 34.3091}, {'id': 106444598, 'name': 'Cool Water', 'type': 'Trad', 'rating': '5.7', 'stars': 2, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Moonshine Dome'], 'url': 'https://www.mountainproject.com/route/106444598/cool-water', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106471474_sqsmall_1494112003.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106471474_small_1494112003.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106471474_smallMed_1494112003.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106471474_medium_1494112003.jpg', 'longitude': -116.8789, 'latitude': 34.3091}, {'id': 106480652, 'name': 'Hootenanny', 'type': 'Trad', 'rating': '5.10a', 'stars': 3.4, 'starVotes': 31, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Coyote Crag'], 'url': 'https://www.mountainproject.com/route/106480652/hootenanny', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106869997_sqsmall_1494147185.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106869997_small_1494147185.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106869997_smallMed_1494147185.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106869997_medium_1494147185.jpg', 'longitude': -116.8781, 'latitude': 34.3078}, {'id': 106521533, 'name': 'Filet of Sole', 'type': 'Trad', 'rating': '5.8', 'stars': 2.7, 'starVotes': 7, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/106521533/filet-of-sole', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/114492305_sqsmall_1529260173.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/114492305_small_1529260173.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/114492305_smallMed_1529260173.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/114492305_medium_1529260173.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 106802720, 'name': 'Triple Decker', 'type': 'Trad', 'rating': '5.7', 'stars': 3, 'starVotes': 10, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/106802720/triple-decker', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 106802728, 'name': 'Colonial Ear Wax', 'type': 'Trad', 'rating': '5.8', 'stars': 3, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/106802728/colonial-ear-wax', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 106809384, 'name': 'Green Goblin', 'type': 'Trad', 'rating': '5.7', 'stars': 3.6, 'starVotes': 27, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/106809384/green-goblin', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108358713_sqsmall_1494278562.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/108358713_small_1494278562.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108358713_smallMed_1494278562.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/108358713_medium_1494278562.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 106809677, 'name': 'Dust in the Wind', 'type': 'Trad', 'rating': '5.5', 'stars': 2.8, 'starVotes': 22, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/106809677/dust-in-the-wind', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106809699_sqsmall_1494142499.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106809699_small_1494142499.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106809699_smallMed_1494142499.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106809699_medium_1494142499.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 106814369, 'name': 'Bobbing for Ear Snax', 'type': 'Trad', 'rating': '5.9', 'stars': 3, 'starVotes': 7, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/106814369/bobbing-for-ear-snax', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 106837289, 'name': 'Bye Chimney', 'type': 'Trad', 'rating': '5.7', 'stars': 2.6, 'starVotes': 7, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/106837289/bye-chimney', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 106837304, 'name': 'Mad Season', 'type': 'Sport', 'rating': '5.11b', 'stars': 2, 'starVotes': 2, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/106837304/mad-season', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106851447_sqsmall_1494145597.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106851447_small_1494145597.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106851447_smallMed_1494145597.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106851447_medium_1494145597.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 106837320, 'name': 'Dos Dose', 'type': 'Trad', 'rating': '5.9', 'stars': 3, 'starVotes': 2, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/106837320/dos-dose', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106837334_sqsmall_1494144485.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106837334_small_1494144485.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106837334_smallMed_1494144485.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106837334_medium_1494144485.jpg', 'longitude': -116.8786, 'latitude': 34.3079}, {'id': 106837985, 'name': 'Crematorium', 'type': 'Sport', 'rating': '5.12d', 'stars': 4, 'starVotes': 8, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Incinerator Wall'], 'url': 'https://www.mountainproject.com/route/106837985/crematorium', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/117535800_sqsmall_1564725517.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/117535800_small_1564725517.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/117535800_smallMed_1564725517.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/117535800_medium_1564725517.jpg', 'longitude': -116.8782, 'latitude': 34.3091}, {'id': 106840238, 'name': 'Lodestone', 'type': 'Sport', 'rating': '5.10b', 'stars': 3.6, 'starVotes': 34, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/106840238/lodestone', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106840590_sqsmall_1494144728.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106840590_small_1494144728.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106840590_smallMed_1494144728.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106840590_medium_1494144728.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 106840256, 'name': 'Golden Gloves', 'type': 'Sport', 'rating': '5.11b', 'stars': 2.3, 'starVotes': 4, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/106840256/golden-gloves', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106840605_sqsmall_1494144729.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106840605_small_1494144729.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106840605_smallMed_1494144729.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106840605_medium_1494144729.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 107224078, 'name': 'Cap Gun', 'type': 'Trad', 'rating': '5.1', 'stars': 2.4, 'starVotes': 11, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/107224078/cap-gun', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 107985763, 'name': 'Long Branch', 'type': 'Sport', 'rating': '5.10a', 'stars': 2.6, 'starVotes': 20, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/107985763/long-branch', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107987007_sqsmall_1494255478.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107987007_small_1494255478.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107987007_smallMed_1494255478.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107987007_medium_1494255478.jpg', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 107987020, 'name': 'Coolie Crank', 'type': 'TR', 'rating': '5.10c', 'stars': 2, 'starVotes': 1, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Bonsai Boulder'], 'url': 'https://www.mountainproject.com/route/107987020/coolie-crank', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/107987032_sqsmall_1494255479.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/107987032_small_1494255479.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/107987032_smallMed_1494255479.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/107987032_medium_1494255479.jpg', 'longitude': -116.8786, 'latitude': 34.308}, {'id': 110964122, 'name': 'Dirty Captain Freeman', 'type': 'Trad', 'rating': '5.10c', 'stars': 2.7, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Gunsmoke Wall'], 'url': 'https://www.mountainproject.com/route/110964122/dirty-captain-freeman', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8785, 'latitude': 34.3082}, {'id': 114534865, 'name': 'Phantom Ore Cart', 'type': 'Sport', 'rating': '5.10b', 'stars': 2.8, 'starVotes': 6, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/114534865/phantom-ore-cart', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/114536936_sqsmall_1530386263.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/114536936_small_1530386263.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/114536936_smallMed_1530386263.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/114536936_medium_1530386263.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 114534895, 'name': "Miner's Milk", 'type': 'Trad', 'rating': '5.9', 'stars': 2.8, 'starVotes': 4, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/114534895/miners-milk', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/114540398_sqsmall_1530469658.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/114540398_small_1530469658.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/114540398_smallMed_1530469658.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/114540398_medium_1530469658.jpg', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 114536109, 'name': 'Miner Forty-Niner', 'type': 'Trad', 'rating': '5.9+ PG13', 'stars': 3, 'starVotes': 3, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Motherlode Rock', 'Motherlode Rock - East Face'], 'url': 'https://www.mountainproject.com/route/114536109/miner-forty-niner', 'imgSqSmall': '', 'imgSmall': '', 'imgSmallMed': '', 'imgMedium': '', 'longitude': -116.8781, 'latitude': 34.3087}, {'id': 114536121, 'name': 'Fire it Up', 'type': 'Sport', 'rating': '5.10a', 'stars': 3.5, 'starVotes': 6, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Incinerator Wall'], 'url': 'https://www.mountainproject.com/route/114536121/fire-it-up', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/114536918_sqsmall_1530386061.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/114536918_small_1530386061.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/114536918_smallMed_1530386061.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/114536918_medium_1530386061.jpg', 'longitude': -116.8782, 'latitude': 34.309}, {'id': 114944809, 'name': 'Whiskey Dave', 'type': 'Sport', 'rating': '5.8', 'stars': 2.8, 'starVotes': 6, 'pitches': 1, 'location': ['California', 'San Bernardino Mountains', 'Big Bear Lake Area', 'Big Bear North', 'Holcomb Valley Pinnacles', 'Central Pinnacles', 'Claim Jumper Wall'], 'url': 'https://www.mountainproject.com/route/114944809/whiskey-dave', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/115325826_sqsmall_1537157224.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/115325826_small_1537157224.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/115325826_smallMed_1537157224.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/115325826_medium_1537157224.jpg', 'longitude': -116.8786, 'latitude': 34.3079}]
# temp_df = pd.DataFrame(a)
# temp_df
df = pd.concat(dfs)

In [79]:
df.location.unique()

array(['California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Coyote Crag',
       'California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Doc Holliday Wall',
       'California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Claim Jumper Wall',
       'California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Gunsmoke Wall',
       'California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Incinerator Wall',
       'California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Gold Wall',
       'California-San Bernardino Mountains-Big Bear Lake Area-Big Bear North-Holcomb Valley Pinnacles-Central Pinnacles-Mad Cow Wall',
       'California-San Bernardino Mou

In [0]:
csv_string = 'Latitude,Longitude,Name,Description'

locs = df.location.unique()
groups = df.groupby('location')

In [0]:
for loc in locs:
    group = groups.get_group(loc)
    lat,long = group.iloc[0].latitude, group.iloc[0].longitude
    cmt = ('   '.join(group.route.to_list())).replace(',','')  # comment is a string list of routes separrated by double underscore. remove commas just in case
    name = (group.iloc[0].location.split('-')[-1]).replace(',','') # start with newline char, remove commas just in case
    newline = ','.join(['\n'+str(lat),str(long),name,cmt])
    csv_string += newline

In [103]:
print(csv_string)

Latitude,Longitude,Name,Description
34.3033,-116.8795,Coyote Crag,Bye Crackie-Sport-5.7-1p   Coyotes at Sunset-Sport-5.8-1p   Black Magic Poodle-Sport-5.9-1p   Golden Poodle-Sport-5.9+-1p   Golden Spike-Sport-5.10b-1p   High Noon-Sport-5.10b-1p   Pass the Bucket-Trad-5.8 PG13-1p   Es Muy Bueno-Trad-5.8-1p   Western Farm Service-Trad-5.8-1p   Red Brewster-Trad-5.9 R-1p   Eight Second Ride-Sport-5.8-1p   Saddle Tramp-TR-5.9-1p   Hootenanny-Trad-5.10a-1p
34.3078,-116.8783,Doc Holliday Wall,Pistol Pete-Sport-5.10a/b-1p   Doc's Holiday-Sport-5.10d-1p   Unforgiven-Sport-5.11b-1p   Quick on the Draw-Sport-5.10d-1p   Far Beyond Driven-Sport-5.10b-1p   Loose Women Loose Rock-Trad-5.9+-1p   Skyline Pillar-Sport-5.7-1p
34.3079,-116.8786,Claim Jumper Wall,Claim Jumper-Sport-5.10a-1p   One Armed Bandit-Sport-5.10a-1p   Better Luck Next Time-Sport-5.10d-1p   Lady Luck-Sport-5.9-1p   Lucky Cuss-Sport-5.11a-1p   Coyotes in the Henhouse-Sport-5.10d-1p   Brewed Awakening-Trad-5.11b-1p   The Papper-Sport

In [0]:
filename = input ("what do you want to name this?")

filename = filename.replace(' ','_')

csv_filename = filename+'.csv'
gpx_filename = filename+'.gpx'


csv_file = open(csv_filename,'w')
csv_file.write(csv_string)
csv_file.close()


create_command = "gpsbabel -i csv -f "+csv_filename+" -o gpx -F "+gpx_filename
remove_command = "rm "+csv_filename
os.system(create_command)
os.system(remove_command)
